# The second  part in the series (MLP -> Multilayer perceptron)

https://youtu.be/TCH_1BHY58I

What I will try to do in this notebook is rather trying to use Pytorch to train something

The guy in the video basically is going to create an embedding (encoder -> decoder) network.

In [1]:
print("Hello world!")

Hello world!
